In [130]:
import numpy as np

In [92]:
def get_scorer(row_1_bias: int = 0.5):
    
    weights = np.array([row_1_bias/3*2, (1 - row_1_bias)/3*2, 1/3])

    def score(succsesses: np.ndarray, remaining: np.ndarray, choice: int, chance: float):

        if(np.sum(remaining) <= 0):
            res = (succsesses * weights).sum()
            return res
        
        if(remaining[choice] <= 0):
            return -100


        suc = succsesses.copy()
        suc[choice] += 1
        fail = succsesses
        
        remaining[choice] -= 1

        scores = np.array([0, 0, 0])
        for i in range(3):
            scores[i] = chance * score(suc.copy(), remaining.copy(), i, down(chance)) + (1 - chance) * score(fail.copy(), remaining.copy(), i, up(chance))


        return np.max(scores)


    return score



s = np.full(3, 0, int)
r = np.full(3, 3, int)
chance = 0.75


In [81]:
row = np.random.choice(np.where(r > 0)[0])
hit = int(np.random.uniform() < chance)
hit

1

In [129]:
def random_complete(succsesses: np.ndarray, remaining: np.ndarray, chance: float):
    if sum(remaining) <= 0:
        return succsesses

    row = np.random.choice(np.where(remaining > 0)[0])
    hit = int(np.random.uniform() < chance)

    succsesses[row] += hit
    if hit:
        chance = max(0.25, chance - 0.1)
    else:
        chance = min(0.75, chance + 0.1)

    remaining[row] -= 1

    return random_complete(succsesses, remaining, chance)




a = random_complete(s.copy(), r.copy(), chance)

118 µs ± 5.74 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [158]:
from time import perf_counter as timer


def random_complete_vek(arr):
    succsesses = arr[0:3]
    remaining = arr[3:6]
    chance = arr[6]
    return random_complete(succsesses, remaining, chance)


def vek(suc, rem, chance, length = 1000):
    arr = np.hstack((suc, rem, chance))
    return np.repeat(arr[None,:], length, 0)


s = np.full(3, 0, int)
r = np.full(3, 3, int)
chance = 0.75

arr = vek(s, r, chance, length=10000)

np.apply_along_axis(random_complete_vek, 1, arr)

0.00018519999957788968
1.3204909999994925
1.3491253000001961


1.6983376999996835